In [1]:
import sys

sys.path.append('/scratch/gpfs/mb5157/repos/graphmert/transformers/src')

In [2]:
import os
import sys
import json
import re
import nest_asyncio, asyncio
import logging
from pathlib import Path
import pyarrow as pa

from datasets import Dataset, Features, Sequence, Value, load_from_disk, concatenate_datasets
from transformers import AutoTokenizer
from google import genai
from google.genai import types

/scratch/gpfs/mb5157/CONDA/envs/graphmert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise RuntimeError("GEMINI_API_KEY env var not set – please `export GEMINI_API_KEY=…` before launching.")
    
MODEL = "gemini-2.0-flash"
model_name = "gemini-2.0"

tokenizer = AutoTokenizer.from_pretrained(
    "/scratch/gpfs/mb5157/tokenizers/biomedbert_fast_tokenizer"
)


SAVE_CHUNK      = 1000               # save result each save every BATCH_SIZE examples
PRINTOUT        = True
TAKE_SUBSET     = False
SUBSET_SIZE      = 50
BATCH_SIZE      = 128               # how many concurrent Gemini calls
TIME_SLEEP = 0.05

TTL = '259200s' # for cache
PRINT_RAW_RESPONSE = False

# !!!!!!!!! update ALLOWED_RELATIONS_GEMINI before running

In [5]:
logging.basicConfig(level=logging.INFO, force=True)

logger = logging.getLogger(__name__)

logger.setLevel(logging.INFO)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
fmt = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
handler.setFormatter(fmt)
logger.addHandler(handler)

# Quiet down other noisy loggers
logging.getLogger("google_genai.models").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [40]:
from importlib import reload

import prompt_library.relation_match_prompts as prompts_module
reload(prompts_module)

from prompt_library.relation_match_prompts import (
    SYSTEM_CONTEXT, 
    example_user_1, example_assistant_1, example_explanation_1,
    example_user_2, example_assistant_2, example_explanation_2,
    example_user_3, example_assistant_3, example_explanation_3,
    example_user_4, example_assistant_4, example_explanation_4,
    example_user_5, example_assistant_5, example_explanation_5,
)


few_shot_examples = [
    {"user_query": example_user_1, "model_answer": example_assistant_1, "model_explanation": example_explanation_1},
    {"user_query": example_user_2, "model_answer": example_assistant_2, "model_explanation": example_explanation_2},
    {"user_query": example_user_3, "model_answer": example_assistant_3, "model_explanation": example_explanation_3},
    {"user_query": example_user_4, "model_answer": example_assistant_4,  "model_explanation": example_explanation_4},
    {"user_query": example_user_5, "model_answer": example_assistant_5,  "model_explanation": example_explanation_5},
]

STRUCTURED_CONTENT_FEW_SHOTS = [
    {"role": "user", "parts": [{"text": "I will provide you with examples"}]},
    {"role": "model", "parts": [{"text": "Understood — send the sample and I’ll output relations"}]},
]

for ex in few_shot_examples:
    STRUCTURED_CONTENT_FEW_SHOTS.append({
        "role": "user",
        "parts": [{"text": ex["user_query"]}]
    })
    # example model answer
    STRUCTURED_CONTENT_FEW_SHOTS.append({
        "role": "model",
        "parts": [{"text": ex["model_answer"]}]
    })

    if "model_explanation" in ex:
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "user",
            "parts": [{"text": "Explanation of the previous output:"}]
        })
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "model",
            "parts": [{"text": ex["model_explanation"]}]
        })

    if "model_explanation_negative" in ex:
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "user",
            "parts": [{"text": "Explanation of what is wrong with the previous output:"}]
        })
        STRUCTURED_CONTENT_FEW_SHOTS.append({
            "role": "model",
            "parts": [{"text": ex["model_explanation_negative"]}]
        })

STRUCTURED_CONTENT_FEW_SHOTS.append({
    "role": "user",
    "parts": [{"text": "**End of examples**.\nNow read the actual input:"}]
})

In [7]:
def extract_answer(response: str) -> str:
    """
    Extracts the rightmost JSON object from an LLM response string
    and returns it as a compact JSON string.
    """
    # 1) Remove any Markdown fence lines (```...```)
    response = re.sub(r'(?m)^```.*\n?', "", response)

    # 2) Find all {...} spans across lines
    matches = re.findall(r"\{.*?\}", response, flags=re.DOTALL)
    if not matches:
        return ""  # No JSON object found

    # 3) Use the last (rightmost) match
    candidate = matches[-1]

    # 4) Validate and return compact JSON
    try:
        obj = json.loads(candidate)
        return json.dumps(obj, separators=(',', ':'))
    except json.JSONDecodeError:
        return ""

In [44]:
async def gemini_call(client, user_query: str, count_only, cache=None, cache_system_prompt=True, max_retries: int=3) -> str:
    """
    Wrap Gemini call in a retrying try/except so it never blows up.
    count_only: mock call for counting input tokens.
    """
    global input_tokens, output_tokens
    # count_tokens is mock request, generate_content is actual paid request
    call_fn = client.aio.models.count_tokens if count_only else client.aio.models.generate_content

    config = types.GenerateContentConfig(cached_content=cache.name) if cache is not None else None
    if cache_system_prompt:
        contents = [{
                "role": "user",
                "parts": [{"text": user_query}]
            }]
    else:
        contents = []
        contents.extend(STRUCTURED_CONTENT_FEW_SHOTS)
        contents.append({
                "role": "user",
                "parts": [{"text": user_query}]
            })
        
    for attempt in range(1, max_retries + 1):
        try:            
            resp = await call_fn(model=MODEL, contents=contents, config=config)
            if count_only:
                # count_tokens returns an int of prompt tokens
                input_tokens += resp.total_tokens
                return ""

            # real response path
            metadata = resp.usage_metadata
            pt = metadata.prompt_token_count or 0
            cc = metadata.cached_content_token_count or 0
            ct = metadata.candidates_token_count or 0
            input_tokens += pt - cc
            output_tokens += ct
            return getattr(resp, "text", "") or ""

        except Exception as e:
            # catch HTTPError, ValidationError, ServerError, etc.
            logger.warning(f"Gemini call attempt {attempt}/{max_retries} failed: {e!r}")
            if attempt < max_retries:
                await asyncio.sleep(0.5 * attempt)    # simple backoff
            else:
                logger.error(f"All {max_retries} Gemini attempts failed for query: {user_query[:50]!r}")
                return ""  # give up


async def process_batch(client, cache, slice_of_examples, count_only, cache_system_prompt, print_raw_response=PRINT_RAW_RESPONSE):
    """
    Fire off up to BATCH_SIZE concurrent requests.
    """
    tasks = []
    for ex in slice_of_examples:
        heads = ex['head_positions']
        heads = json.loads(heads)
        heads = list(heads.keys())
        if len(heads) != 0:
            seq  = tokenizer.decode(ex["input_ids"], skip_special_tokens=True)
            uq = f"Input:\n{seq}\n\nheads: {heads}"
            
            tasks.append(gemini_call(client, uq, count_only, cache, cache_system_prompt))
        else:
            tasks.append(asyncio.sleep(0, result=""))
        await asyncio.sleep(TIME_SLEEP)
        
    raw_resps = await asyncio.gather(*tasks)

    if print_raw_response:
        for raw_resp in raw_resps:
            print(raw_resp)
    
    cleaned = [extract_answer(r) for r in raw_resps]
    return cleaned


async def create_cache(client, count_only):
    global input_tokens, cached_tokens

    if count_only: # free, only for input token count
        cache_contents_mock = [{"role": "user", "parts": [{"text": SYSTEM_CONTEXT}]}]
        cache_contents_mock.extend(STRUCTURED_CONTENT_FEW_SHOTS)
        
        cache_call_tokens = await client.aio.models.count_tokens(
            model=MODEL,
            contents=cache_contents_mock,
        )
        cached_tokens += cache_call_tokens.total_tokens
        return None
        
    else: # paid call
        cache_config = types.CreateCachedContentConfig(
                system_instruction=SYSTEM_CONTEXT,
                contents=STRUCTURED_CONTENT_FEW_SHOTS,
                ttl=TTL
            )
        cache = await client.aio.caches.create(
            model=MODEL,
            config=cache_config,  
        )
        logger.info("created cache")
        # count actual tokens cached in a paid request 
        cached_tokens_call = cache.usage_metadata.total_token_count
        cached_tokens += cached_tokens_call
        return cache


async def run_all(client, dataset: Dataset, chunk_start: int, chunk_end: int, cache=None, count_only=True):
    """
    Split into batches
    """
    n = len(dataset)
    all_tails = [None] * n

    for start in range(0, n, BATCH_SIZE):
        end = min(start + BATCH_SIZE, n)
        slice_ds = dataset.select(range(start, end))
        cleaned  = await process_batch(client, cache, slice_ds, count_only, cache_system_prompt)
        all_tails[start:end] = cleaned
        if PRINTOUT:
            logger.info(f"Gemini: processed {end}/{n}")
            
    return all_tails

In [9]:
def output_sample(dataset, start=0):
    end = start + 100
    items = dataset # [start: end]
    for i, values in \
                enumerate(zip(items['input_ids'], items['head'], \
                              items['relation'], items['predictions'], items['tails'])):
        input_ids, head, relation, predictions, tails = values
        if head != '{}':
            print(i + start)
            print(tokenizer.decode(input_ids), '\n')
            print('head:', head)
            print('relation:', relation)
            print('predictions:', predictions)
            print('tails:', tails)
            print()

In [10]:
async def main(dataset_dir: str, path_to_save, count_only=True, cache_system_prompt=True, start_idx: int = 0, end_idx=None):
    ds_full = Dataset.load_from_disk(str(dataset_dir))
    if TAKE_SUBSET:
        overall_end = min(start_idx + SUBSET_SIZE, len(ds_full))
    else:
        overall_end = len(ds_full) if end_idx is None else min(len(ds_full), end_idx)

    client = genai.Client(api_key=GEMINI_API_KEY)
    cache = None

    try:
        if cache_system_prompt:
            cache = await create_cache(client, count_only)
            
        for chunk_start in range(start_idx, overall_end, SAVE_CHUNK):
            chunk_end = min(chunk_start + SAVE_CHUNK, overall_end)
            ds_chunk = ds_full.select(range(chunk_start, chunk_end))
            tails = await run_all(client, ds_chunk, chunk_start, chunk_end, cache, count_only=count_only)
            
            assert len(tails) == ds_chunk.num_rows, f"{len(tails)=} vs {ds_chunk.num_rows=}"
            if not count_only:
                # coerce any stray string into a single‐element list
                fixed = [[str(x) for x in t] if isinstance(t, list) else [str(t)] for t in tails]
                tails_arr = pa.array(fixed, type=pa.list_(pa.string()))
                ds_chunk = ds_chunk.add_column("response", fixed)
                if TAKE_SUBSET:
                    save_name = f"{model_name}_subset_{SUBSET_SIZE}_{chunk_start}-{chunk_end}"
                else:
                    save_name = f"{model_name}_{chunk_start}-{chunk_end}"
        
                out_dir = os.path.join(str(path_to_save), save_name)
                ds_chunk.save_to_disk(out_dir)
                logger.info(f"Saved chunk {chunk_start}-{chunk_end} to {out_dir}")
    finally:
        if cache is not None:
            await client.aio.caches.delete(name=cache.name)

In [48]:
input_tokens = 0
output_tokens = 0
cached_tokens = 0

count_only = False # True means no real requests
cache_system_prompt = True

dataset_dir = Path('/scratch/gpfs/mb5157/repos/graphmert/graphmert/datasets/dataset_with_heads/350k_abstracts_gemini_heads_all')
path_to_save = Path('/scratch/gpfs/mb5157/repos/graphmert/graphmert/datasets/dataset_with_heads/350k_abstracts_gemini_relations')
logger.info(f'request with {model_name}')
await main(dataset_dir, path_to_save, count_only=count_only, cache_system_prompt=cache_system_prompt, start_idx=0, end_idx=None)

print('===finished===')
print('input_tokens:', input_tokens)
print('output_tokens:', output_tokens)
print('cached_tokens', cached_tokens)

2025-08-09 13:56:04,090 INFO request with gemini-2.0


INFO:__main__:request with gemini-2.0


2025-08-09 13:56:04,776 INFO created cache


INFO:__main__:created cache
/scratch/gpfs/mb5157/CONDA/envs/graphmert/lib/python3.12/site-packages/datasets/formatting/formatting.py:145: RuntimeWarning: coroutine 'sleep' was never awaited
  return _unnest(pa_table.to_pydict())
/scratch/gpfs/mb5157/CONDA/envs/graphmert/lib/python3.12/site-packages/datasets/formatting/formatting.py:145: RuntimeWarning: coroutine 'gemini_call' was never awaited
  return _unnest(pa_table.to_pydict())


2025-08-09 13:56:13,466 INFO Gemini: processed 128/1000


...

2025-08-10 09:15:56,580 INFO Gemini: processed 666/666


INFO:__main__:Gemini: processed 666/666
Saving the dataset (1/1 shards): 100%|██████████| 666/666 [00:00<00:00, 54303.11 examples/s]

2025-08-10 09:15:56,602 INFO Saved chunk 989000-989666 to /scratch/gpfs/mb5157/repos/graphmert/graphmert/datasets/dataset_with_heads/350k_abstracts_gemini_relations/gemini-2.0_989000-989666



INFO:__main__:Saved chunk 989000-989666 to /scratch/gpfs/mb5157/repos/graphmert/graphmert/datasets/dataset_with_heads/350k_abstracts_gemini_relations/gemini-2.0_989000-989666


===finished===
input_tokens: 85396353
output_tokens: 30150205
cached_tokens 4114
